# Load test data

In [46]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
## old version
# import whisper

# model = whisper.load_model("base")

# audio_path = "data/test/f57c791f-074f-4706-94ca-a9b4bcab7ce4.amr"

# # load audio and pad/trim it to fit 30 seconds
# audio = whisper.load_audio(audio_path)
# audio = whisper.pad_or_trim(audio)

# # make log-Mel spectrogram and move to the same device as the model
# mel = whisper.log_mel_spectrogram(audio).to(model.device)

# # detect the spoken language
# _, probs = model.detect_language(mel)
# print(f"Detected language: {max(probs, key=probs.get)}")

# # decode the audio
# options = whisper.DecodingOptions()
# result = whisper.decode(model, mel, options)

# # print the recognized text
# print(result.text)

# def whisper_decode(path):
#     # load audio and pad/trim it to fit 30 seconds
#     audio = whisper.load_audio(path)
#     audio = whisper.pad_or_trim(audio)

#     # make log-Mel spectrogram and move to the same device as the model
#     mel = whisper.log_mel_spectrogram(audio).to(model.device)

#     # detect the spoken language
#     _, probs = model.detect_language(mel)
#     # print(f"Detected language: {max(probs, key=probs.get)}")

#     # decode the audio
#     options = whisper.DecodingOptions()
#     result = whisper.decode(model, mel, options)

#     # print the recognized text
#     # print(result.text)
    
#     return {'text': result.text, 'language': max(probs, key=probs.get)}

In [2]:
import os

test_data_dir = "data/test"

filenames = [filename for filename in os.listdir(test_data_dir) if filename.endswith('.amr')]
print(filenames)

['a8f51c60-5a4a-4765-aa79-638f76d2f9b2.amr', 'c40afd4e-dafb-40a5-8c6e-ce9283a33278.amr', '94a7e398-202d-4bcc-b991-81eced1c892f.amr', '19a69c77-0be3-49d5-8d0d-90fea3c7a1c4.amr', '7997ef05-06ce-4389-a915-c291782ae901.amr', '4a4115de-00b5-4cf9-b08b-f147d0ec9a75.amr', 'c124f9b4-38e1-4cee-8b41-49f57f069806.amr', '5aebf10f-21c0-438f-9b81-4be299508517.amr', 'f57c791f-074f-4706-94ca-a9b4bcab7ce4.amr', 'c539ea2b-dd7a-4358-b9f2-19545610c01b.amr', '17f3f07c-eca6-41d8-a3e4-9c0120595eb3.amr', '13717e94-e8ca-4d65-be27-ed97587c5057.amr', '6ae92f35-3139-417c-b0f4-d30ac25b3937.amr', '87666cfe-99e9-41c6-a750-52c5f1239b97.amr', '9670e5ad-3c33-444e-bba8-7c742f4ef7f8.amr', 'f3319895-e3e0-4cb9-a60a-4bda01beb448.amr', 'aba769fe-5f26-412b-8a5d-3d79f3ef6857.amr', '57ccf65f-b8ad-4b7e-a813-f303be7b0d10.amr', 'a03fa709-ce7b-42c0-b29f-4443e0e6c770.amr', 'c6d50f43-390b-4493-ab04-2f0bda10c83b.amr', '51ab811e-12a0-47b6-834d-939b8b4ec144.amr', '8ab13ab7-a573-4ac1-b1bf-0b229a19cca6.amr', '5c01bb8a-e3ca-4888-94d4-43c09f

In [4]:
# import pandas as pd
# from tqdm import tqdm

# results = []

# for filename in tqdm(filenames):
#     result = whisper_decode(os.path.join(test_data_dir, filename))
#     result['filename'] = filename
#     results.append(result)

# df_pred = pd.DataFrame(results)
# df_pred.head()

In [5]:
# df_true = pd.read_csv("./data/语料.csv")

# df_all = pd.merge(df_true, df_pred, left_on='record_file_id', right_on='filename')

In [6]:
# df_all.head()

In [7]:
# df_all.to_csv("outputs/combined_results.csv", index=False)

# whisper large

In [8]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"device: {device}")

model_id = "/home/ec2-user/SageMaker/efs/Models/whisper-medium"
model_id = "/home/ec2-user/SageMaker/efs/Models/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, use_flash_attention_2=True)

model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

sample = os.path.join(test_data_dir, filenames[0])

result = pipe(sample)
print(result["text"])

/home/ec2-user/SageMaker/efs/conda_envs/whisper_py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device: cuda:0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/ec2-user/SageMaker/efs/conda_envs/whisper_py310/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


CMSRO20-DICR


In [ ]:
import pandas as pd
from tqdm import tqdm
import time

results = []

for filename in tqdm(filenames):
    start_time = time.time()
    result = pipe(os.path.join(test_data_dir, filename), generate_kwargs={"language": "cantonese"})
    time_cost = time.time() - start_time
    result['filename'] = filename
    result['time_cost'] = round(time_cost, 2)
    results.append(result)

df_pred = pd.DataFrame(results)
df_pred.head()

In [ ]:
## convert whisper-large-v3 to faster-whisper-large-v3

# !ct2-transformers-converter --model openai/whisper-large-v3 --output_dir faster-whisper-large-v3 \
#     --copy_files tokenizer.json preprocessor_config.json --quantization float16

## faster_whisper_v3

In [ ]:
from faster_whisper import WhisperModel

model_id_faster =  "/home/ec2-user/SageMaker/efs/Models/faster-whisper-large-v3"
model = WhisperModel(model_id_faster)

segments, info = model.transcribe(sample, language="cantonese")
print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    print()


In [27]:
df_true = pd.read_csv("./data/语料.csv")

df_all = pd.merge(df_true, df_pred, left_on='record_file_id', right_on='filename')

In [28]:
df_all.head(10)

,category_name,content,user_id,record_file_id,text,chunks,filename
0,Model,CMSN 20 si,wuzy37,8deb96cd-1939-46d0-8f87-5bd798ade923.amr,新年快樂,"[{'timestamp': (0.0, 3.1), 'text': ' 新年快樂'}]",8deb96cd-1939-46d0-8f87-5bd798ade923.amr
1,Model,CMSRO 20di cr,wuzy37,09b311d8-c557-4bfb-bac5-f0d18a5da8e9.amr,《天使的名字》,"[{'timestamp': (0.0, 7.78), 'text': ' 《天使的名字》'}]",09b311d8-c557-4bfb-bac5-f0d18a5da8e9.amr
2,Model,CMSN 20 si,ex_xietian2,6ae92f35-3139-417c-b0f4-d30ac25b3937.amr,測試,"[{'timestamp': (0.0, 0.96), 'text': ' 測試'}]",6ae92f35-3139-417c-b0f4-d30ac25b3937.amr
3,Model,CMSRO 20di cr,ex_xietian2,3b65f69e-5aea-4211-acaf-b7bb2f651755.amr,CNSRO20DICR,"[{'timestamp': (0.0, 5.28), 'text': ' CNSRO20D...",3b65f69e-5aea-4211-acaf-b7bb2f651755.amr
4,Model,CMSRO 20di rd,ex_xietian2,c40afd4e-dafb-40a5-8c6e-ce9283a33278.amr,自動化測試CNNX-R5-20DR2D,"[{'timestamp': (0.0, 4.94), 'text': ' 自動化測試CNN...",c40afd4e-dafb-40a5-8c6e-ce9283a33278.amr
5,Model,AF-52CS1TRHK(H),ex_xietian2,8ab13ab7-a573-4ac1-b1bf-0b229a19cca6.amr,AF-52CSE TRH Key,"[{'timestamp': (0.0, 5.66), 'text': ' AF-52CSE...",8ab13ab7-a573-4ac1-b1bf-0b229a19cca6.amr
6,Model,CMSN 20 si,yeungchimkuen,2956c101-d82f-4f2c-8f1b-6119fa87694d.amr,CMSN20SI,"[{'timestamp': (0.0, 3.08), 'text': ' CMSN20SI'}]",2956c101-d82f-4f2c-8f1b-6119fa87694d.amr
7,Model,CMSRO 20di cr,yeungchimkuen,c124f9b4-38e1-4cee-8b41-49f57f069806.amr,CMSRO20DICR,"[{'timestamp': (0.0, 4.08), 'text': ' CMSRO20D...",c124f9b4-38e1-4cee-8b41-49f57f069806.amr
8,Model,CMSRO 20di rd,yeungchimkuen,17f3f07c-eca6-41d8-a3e4-9c0120595eb3.amr,CMSRO20-DIRD,"[{'timestamp': (0.0, 4.02), 'text': ' CMSRO20-...",17f3f07c-eca6-41d8-a3e4-9c0120595eb3.amr
9,Model,AF-52CS1TRHK(H),yeungchimkuen,5102309c-8fb3-4232-b257-2bd8ed6ef53e.amr,AS-52CS1TRHKQH,"[{'timestamp': (0.0, 6.06), 'text': ' AS-52CS1...",5102309c-8fb3-4232-b257-2bd8ed6ef53e.amr


# Calculate Character Error Rate (WER)

In [41]:
import re

class ChineseTextNormalizer:
    def __init__(self, split_letters: bool = False):

        self.split_letters = split_letters

    def __call__(self, s: str):
        s = s.lower()
        # s = re.sub(r"[<\[][^>\]]*[>\]]", "", s)  # remove words between brackets
        # s = re.sub(r"\(([^)]+?)\)", "", s)  # remove words between parenthesis
        # s = self.clean(s).lower()

        # if self.split_letters:
        #     s = " ".join(regex.findall(r"\X", s, regex.U))

        s = re.sub(
            r"\s+", "", s
        )  # remove any successive whitespace characters
        
        return s

In [42]:
import jiwer
from whisper.normalizers import BasicTextNormalizer, EnglishTextNormalizer

# normalizer = EnglishTextNormalizer()
# normalizer = BasicTextNormalizer()
normalizer = ChineseTextNormalizer()

In [43]:
for i, row in tqdm(df_all.iterrows(), total=df_all.shape[0]):
    
    normalized_content = normalizer(row['content'])
    normalized_text = normalizer(row['text'])
    
    df_all.loc[i, 'normalized_content'] = normalized_content
    df_all.loc[i, 'normalized_text'] = normalized_text

new_columns = ['category_name', 'user_id', 'content', 'normalized_content', 'text', 'normalized_text', 'record_file_id', 'chunks', 'filename']
df_all = df_all[new_columns]
df_all.head(10)

100%|██████████| 37/37 [00:00<00:00, 3047.17it/s]


,category_name,user_id,content,normalized_content,text,normalized_text,record_file_id,chunks,filename
0,Model,wuzy37,CMSN 20 si,cmsn20si,新年快樂,新年快樂,8deb96cd-1939-46d0-8f87-5bd798ade923.amr,"[{'timestamp': (0.0, 3.1), 'text': ' 新年快樂'}]",8deb96cd-1939-46d0-8f87-5bd798ade923.amr
1,Model,wuzy37,CMSRO 20di cr,cmsro20dicr,《天使的名字》,《天使的名字》,09b311d8-c557-4bfb-bac5-f0d18a5da8e9.amr,"[{'timestamp': (0.0, 7.78), 'text': ' 《天使的名字》'}]",09b311d8-c557-4bfb-bac5-f0d18a5da8e9.amr
2,Model,ex_xietian2,CMSN 20 si,cmsn20si,測試,測試,6ae92f35-3139-417c-b0f4-d30ac25b3937.amr,"[{'timestamp': (0.0, 0.96), 'text': ' 測試'}]",6ae92f35-3139-417c-b0f4-d30ac25b3937.amr
3,Model,ex_xietian2,CMSRO 20di cr,cmsro20dicr,CNSRO20DICR,cnsro20dicr,3b65f69e-5aea-4211-acaf-b7bb2f651755.amr,"[{'timestamp': (0.0, 5.28), 'text': ' CNSRO20D...",3b65f69e-5aea-4211-acaf-b7bb2f651755.amr
4,Model,ex_xietian2,CMSRO 20di rd,cmsro20dird,自動化測試CNNX-R5-20DR2D,自動化測試cnnx-r5-20dr2d,c40afd4e-dafb-40a5-8c6e-ce9283a33278.amr,"[{'timestamp': (0.0, 4.94), 'text': ' 自動化測試CNN...",c40afd4e-dafb-40a5-8c6e-ce9283a33278.amr
5,Model,ex_xietian2,AF-52CS1TRHK(H),af-52cs1trhk(h),AF-52CSE TRH Key,af-52csetrhkey,8ab13ab7-a573-4ac1-b1bf-0b229a19cca6.amr,"[{'timestamp': (0.0, 5.66), 'text': ' AF-52CSE...",8ab13ab7-a573-4ac1-b1bf-0b229a19cca6.amr
6,Model,yeungchimkuen,CMSN 20 si,cmsn20si,CMSN20SI,cmsn20si,2956c101-d82f-4f2c-8f1b-6119fa87694d.amr,"[{'timestamp': (0.0, 3.08), 'text': ' CMSN20SI'}]",2956c101-d82f-4f2c-8f1b-6119fa87694d.amr
7,Model,yeungchimkuen,CMSRO 20di cr,cmsro20dicr,CMSRO20DICR,cmsro20dicr,c124f9b4-38e1-4cee-8b41-49f57f069806.amr,"[{'timestamp': (0.0, 4.08), 'text': ' CMSRO20D...",c124f9b4-38e1-4cee-8b41-49f57f069806.amr
8,Model,yeungchimkuen,CMSRO 20di rd,cmsro20dird,CMSRO20-DIRD,cmsro20-dird,17f3f07c-eca6-41d8-a3e4-9c0120595eb3.amr,"[{'timestamp': (0.0, 4.02), 'text': ' CMSRO20-...",17f3f07c-eca6-41d8-a3e4-9c0120595eb3.amr
9,Model,yeungchimkuen,AF-52CS1TRHK(H),af-52cs1trhk(h),AS-52CS1TRHKQH,as-52cs1trhkqh,5102309c-8fb3-4232-b257-2bd8ed6ef53e.amr,"[{'timestamp': (0.0, 6.06), 'text': ' AS-52CS1...",5102309c-8fb3-4232-b257-2bd8ed6ef53e.amr


In [45]:
cer = jiwer.cer(list(df_all["normalized_content"]), list(df_all["normalized_text"]))

print(f"CER: {cer * 100:.2f} %")

CER: 27.94 %
